In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install awscli -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  docutils-common fonts-droid-fallback fonts-noto-mono fonts-urw-base35
  ghostscript groff gsfonts imagemagick imagemagick-6-common imagemagick-6.q16
  libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9 libgs9-common
  libidn12 libijs-0.35 libimagequant0 libjbig2dec0 libjxr-tools libjxr0
  liblqr-1-0 libmagickcore-6.q16-6 libmagickcore-6.q16-6-extra
  libmagickwand-6.q16-6 libnetpbm10 libraqm0 libwmflite-0.2-7 netpbm
  poppler-data psutils python3-botocore python3-certifi python3-chardet
  python3-colorama python3-dateutil python3-docutils python3-idna
  python3-jmespath python3-olefile python3-pil python3-pyasn1 python3-pygments
  python3-requests python3-roman python3-rsa python3-s3transfer
  python3-urllib3 python3-yaml sgml-base xml-core
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  gh

In [ ]:
!pip install --upgrade pip
!pip install --upgrade urllib3==1.26.16 pyOpenSSL

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 76.0 MB/s  0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: pyOpenSSL
    Found existing installation: pyOpenSSL 24.2.1
    Uninstalling pyOpenSSL-24.2.1:
      Successfully uninstalled pyOpenSSL-24.2.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pyOpenSSL]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the f

In [ ]:
!aws --version

/bin/bash: line 1: aws: command not found


In [ ]:
import subprocess
import os
import json
import pandas as pd
import glob

# Load extracted location IDs
with open("extracted_locations.json", "r") as f:
    locations = json.load(f)

def download_location_years(location_ids, start_year, end_year, output_dir="/content/openaq_merged"):
    os.makedirs(output_dir, exist_ok=True)

    for loc in location_ids:
        loc_id = loc["id"]
        loc_name = loc.get("name", f"loc_{loc_id}")
        print(f"\n=== Processing location: {loc_id} ({loc_name}) ===")

        for year in range(start_year, end_year + 1):
            s3_path = f"s3://openaq-data-archive/records/csv.gz/locationid={loc_id}/year={year}/"
            local_raw_dir = f"/content/openaq_raw/locationid={loc_id}/year={year}/"
            merged_output_path_parquet = f"{output_dir}/locationid={loc_id}_year={year}.parquet"
            merged_output_path_csv = f"{output_dir}/locationid={loc_id}_year={year}.csv"

            # Skip if merged file already exists
            if os.path.exists(merged_output_path_parquet) or os.path.exists(merged_output_path_csv):
                print(f"Merged data for location {loc_id}, year {year} already exists. Skipping download and merge.")
                continue

            # Check if folder exists on S3
            result = subprocess.run(
                ["aws", "s3", "ls", s3_path, "--no-sign-request"],
                capture_output=True, text=True
            )

            if result.stdout.strip() == "":
                print(f"❌ No data for location {loc_id} in {year}")
                continue

            # Download recursively
            print(f"✅ Downloading data for location {loc_id}, year {year} to {local_raw_dir}")
            os.makedirs(local_raw_dir, exist_ok=True)
            subprocess.run([
                "aws", "s3", "cp", "--recursive", "--no-sign-request", s3_path,
                local_raw_dir
            ])

            # Merge CSVs and save as Parquet
            print(f"Merging CSVs for location {loc_id}, year {year} and saving as Parquet")
            all_files = glob.glob(os.path.join(local_raw_dir, "*.csv.gz"))
            if not all_files:
                print(f"No CSV files found for location {loc_id}, year {year}. Skipping merge.")
                continue

            try:
                df = pd.concat((pd.read_csv(f, compression='gzip', low_memory=False) for f in all_files), ignore_index=True)
                df.to_parquet(merged_output_path_parquet)
                print(f"Successfully merged and saved data for location {loc_id}, year {year} to {merged_output_path_parquet}")
            except Exception as e:
                print(f"Error merging and saving data for location {loc_id}, year {year}: {e}")

            # Clean up raw CSVs to save space
            print(f"Cleaning up raw CSV files in {local_raw_dir}")
            subprocess.run(["rm", "-rf", local_raw_dir])


# Example: run for all locations in the file
download_location_years(locations, start_year=2000, end_year=2010)

# Example: run for a subset of locations to test
# download_location_years(locations[:2], start_year=2020, end_year=2024)


=== Processing location: 12 (SPARTAN - IIT Kanpur) ===
❌ No data for location 12 in 2000
❌ No data for location 12 in 2001
❌ No data for location 12 in 2002
❌ No data for location 12 in 2003
❌ No data for location 12 in 2004
❌ No data for location 12 in 2005
❌ No data for location 12 in 2006
❌ No data for location 12 in 2007
❌ No data for location 12 in 2008
❌ No data for location 12 in 2009
❌ No data for location 12 in 2010

=== Processing location: 13 (Delhi Technological University, Delhi - CPCB) ===
❌ No data for location 13 in 2000
❌ No data for location 13 in 2001
❌ No data for location 13 in 2002
❌ No data for location 13 in 2003
❌ No data for location 13 in 2004
❌ No data for location 13 in 2005
❌ No data for location 13 in 2006
❌ No data for location 13 in 2007
❌ No data for location 13 in 2008
❌ No data for location 13 in 2009
❌ No data for location 13 in 2010

=== Processing location: 15 (IGI Airport) ===
❌ No data for location 15 in 2000
❌ No data for location 15 in 2001
❌

In [ ]:
import os
import glob
import pandas as pd

def merge_raw_data_by_year(raw_dir="/content/openaq_raw", output_dir="/content/openaq_merged_yearly"):
    os.makedirs(output_dir, exist_ok=True)
    year_data = {}

    # Iterate through the raw data directory structure
    for location_dir in glob.glob(os.path.join(raw_dir, "locationid=*")):
        for year_dir in glob.glob(os.path.join(location_dir, "year=*")):
            year = os.path.basename(year_dir).split("=")[1]
            all_files_year = glob.glob(os.path.join(year_dir, "**/*.csv.gz"), recursive=True)

            if not all_files_year:
                print(f"No CSV files found for year {year} in {location_dir}. Skipping.")
                continue

            print(f"Processing year {year} in {location_dir}...")

            # Read and concatenate CSVs for the current year
            try:
                df_year = pd.concat((pd.read_csv(f, compression='gzip', low_memory=False) for f in all_files_year), ignore_index=True)
                if year not in year_data:
                    year_data[year] = []
                year_data[year].append(df_year)
            except Exception as e:
                print(f"Error processing year {year} in {location_dir}: {e}")


    # Merge dataframes for each year and save as Parquet
    for year, dfs in year_data.items():
        merged_df = pd.concat(dfs, ignore_index=True)
        output_path = os.path.join(output_dir, f"merged_openaq_{year}.parquet")
        try:
            merged_df.to_parquet(output_path)
            print(f"Successfully merged and saved data for year {year} to {output_path}")
        except Exception as e:
            print(f"Error saving merged data for year {year}: {e}")


# Run the merging function
merge_raw_data_by_year()

Processing year 2019 in /content/openaq_raw/locationid=5622...
Processing year 2018 in /content/openaq_raw/locationid=5622...
Processing year 2016 in /content/openaq_raw/locationid=716...
Processing year 2017 in /content/openaq_raw/locationid=716...
Processing year 2016 in /content/openaq_raw/locationid=594...
Processing year 2017 in /content/openaq_raw/locationid=594...
Processing year 2018 in /content/openaq_raw/locationid=594...
Processing year 2016 in /content/openaq_raw/locationid=2594...
Processing year 2017 in /content/openaq_raw/locationid=2594...
Processing year 2018 in /content/openaq_raw/locationid=2594...
Processing year 2019 in /content/openaq_raw/locationid=5613...
Processing year 2018 in /content/openaq_raw/locationid=5613...
Processing year 2019 in /content/openaq_raw/locationid=5593...
Processing year 2018 in /content/openaq_raw/locationid=5593...
Processing year 2016 in /content/openaq_raw/locationid=378...
Processing year 2017 in /content/openaq_raw/locationid=378...

In [ ]:
import os
import glob
import shutil

def save_to_drive(source_dir="/content/openaq_merged_yearly", destination_dir="/content/drive/MyDrive/openaq_data"):
    os.makedirs(destination_dir, exist_ok=True)
    parquet_files = glob.glob(os.path.join(source_dir, "*.parquet"))

    if not parquet_files:
        print(f"No Parquet files found in {source_dir}")
        return

    for parquet_file in parquet_files:
        file_name = os.path.basename(parquet_file)
        destination_path = os.path.join(destination_dir, file_name)

        if os.path.exists(destination_path):
            print(f"File {file_name} already exists in {destination_dir}. Skipping copy.")
            continue

        try:
            shutil.copy(parquet_file, destination_dir)
            print(f"Copied {parquet_file} to {destination_dir}")
        except Exception as e:
            print(f"Error copying {parquet_file} to {destination_dir}: {e}")

save_to_drive()

File merged_openaq_2017.parquet already exists in /content/drive/MyDrive/openaq_data. Skipping copy.
File merged_openaq_2019.parquet already exists in /content/drive/MyDrive/openaq_data. Skipping copy.
File merged_openaq_2014.parquet already exists in /content/drive/MyDrive/openaq_data. Skipping copy.
File merged_openaq_2013.parquet already exists in /content/drive/MyDrive/openaq_data. Skipping copy.
File merged_openaq_2016.parquet already exists in /content/drive/MyDrive/openaq_data. Skipping copy.
File merged_openaq_2018.parquet already exists in /content/drive/MyDrive/openaq_data. Skipping copy.
File merged_openaq_2015.parquet already exists in /content/drive/MyDrive/openaq_data. Skipping copy.


In [ ]:
import os
import glob
import pandas as pd

def combine_parquet_files(source_dir="/content/drive/MyDrive/openaq_data", output_path="/content/drive/MyDrive/combined_openaq_data.parquet"):
    parquet_files = glob.glob(os.path.join(source_dir, "*.parquet"))

    if not parquet_files:
        print(f"No Parquet files found in {source_dir}")
        return

    print(f"Combining {len(parquet_files)} Parquet files from {source_dir}")

    # Read and concatenate all parquet files
    try:
        combined_df = pd.concat((pd.read_parquet(f) for f in parquet_files), ignore_index=True)
        print("Successfully combined Parquet files.")
    except Exception as e:
        print(f"Error combining Parquet files: {e}")
        return

    # Save the combined dataframe to a new parquet file
    try:
        combined_df.to_parquet(output_path)
        print(f"Successfully saved combined data to {output_path}")
    except Exception as e:
        print(f"Error saving combined data to {output_path}: {e}")

# Run the combine function
combine_parquet_files()

Combining 10 Parquet files from /content/drive/MyDrive/openaq_data
Successfully combined Parquet files.
Successfully saved combined data to /content/drive/MyDrive/combined_openaq_data.parquet


In [ ]:
import pandas as pd

# Load the combined parquet file
combined_df = pd.read_parquet("/content/drive/MyDrive/combined_openaq_data.parquet")

# Convert 'datetime' column to datetime objects
combined_df['datetime'] = pd.to_datetime(combined_df['datetime'])

# Extract year, month, day, and time
combined_df['year'] = combined_df['datetime'].dt.year
combined_df['month'] = combined_df['datetime'].dt.month
combined_df['day'] = combined_df['datetime'].dt.day
combined_df['time'] = combined_df['datetime'].dt.time

# Display the first few rows with new columns
display(combined_df.head())

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,year,month,day,time
0,5622,34957,"NSIT Dwarka, Delhi - CPCB-5622",2022-09-26 00:30:00+05:30,28.60909,77.032541,pm10,µg/m³,102.4,2022,9,26,00:30:00
1,5622,34957,"NSIT Dwarka, Delhi - CPCB-5622",2022-09-26 02:30:00+05:30,28.60909,77.032541,pm10,µg/m³,130.6,2022,9,26,02:30:00
2,5622,34957,"NSIT Dwarka, Delhi - CPCB-5622",2022-09-26 03:30:00+05:30,28.60909,77.032541,pm10,µg/m³,126.0,2022,9,26,03:30:00
3,5622,34957,"NSIT Dwarka, Delhi - CPCB-5622",2022-09-26 04:30:00+05:30,28.60909,77.032541,pm10,µg/m³,99.4,2022,9,26,04:30:00
4,5622,34957,"NSIT Dwarka, Delhi - CPCB-5622",2022-09-26 05:30:00+05:30,28.60909,77.032541,pm10,µg/m³,94.1,2022,9,26,05:30:00


# Task
Use the `combined_df` to create new columns for each unique parameter, including their respective units and values.

## Identify unique parameters

### Subtask:
Identify all unique values in the 'parameter' column of the `combined_df`.


**Reasoning**:
Access the 'parameter' column and find the unique values to identify all unique parameters.



In [ ]:
unique_parameters = combined_df['parameter'].unique()
print(unique_parameters)

['pm10' 'pm25' 'o3' 'co' 'no2' 'so2']


## Pivot the dataframe

### Subtask:
Reshape the DataFrame so that each unique parameter becomes a separate set of columns for its 'value' and 'units'.


**Reasoning**:
Reshape the DataFrame by pivoting it to have parameters as columns with their corresponding values and units.



**Reasoning**:
The pivot_table method defaults to using the mean aggregation function, which fails on the 'units' column because it is of object dtype (strings). I need to explicitly specify an aggregation function that works for both numeric ('value') and object ('units') columns, such as 'first' or 'last', since for a given datetime, location, and parameter, there should only be one value and one unit.



In [ ]:
pivoted_df = combined_df.pivot_table(
    index=['location_id', 'sensors_id', 'location', 'datetime', 'lat', 'lon', 'year', 'month', 'day', 'time'],
    columns='parameter',
    values=['value', 'units'],
    aggfunc='first' # Use 'first' as the aggregation function
)

# Reset the index to make location_id, sensors_id, and location regular columns
pivoted_df = pivoted_df.reset_index()

display(pivoted_df.head())

location_id sensors_id                 location  \
parameter                                                   
0                  12         23  SPARTAN - IIT Kanpur-12   
1                  12         23  SPARTAN - IIT Kanpur-12   
2                  12         23  SPARTAN - IIT Kanpur-12   
3                  12         23  SPARTAN - IIT Kanpur-12   
4                  12         23  SPARTAN - IIT Kanpur-12   

                           datetime     lat     lon  year month day      time  \
parameter                                                                       
0         2013-12-14 16:00:00+05:30  26.519  80.233  2013    12  14  16:00:00   
1         2013-12-14 17:00:00+05:30  26.519  80.233  2013    12  14  17:00:00   
2         2013-12-14 18:00:00+05:30  26.519  80.233  2013    12  14  18:00:00   
3         2013-12-14 19:00:00+05:30  26.519  80.233  2013    12  14  19:00:00   
4         2013-12-14 20:00:00+05:30  26.519  80.233  2013    12  14  20:00:00   

           ... units                  value                          
parameter  ...    o3 pm10   pm25  so2    co no2  o3 pm10   pm25 so2  
0          ...   NaN  NaN  µg/m³  NaN   NaN NaN NaN  NaN  106.5 NaN  
1          ...   NaN  NaN  µg/m³  NaN   NaN NaN NaN  NaN  127.6 NaN  
2          ...   NaN  NaN  µg/m³  NaN   NaN NaN NaN  NaN  124.0 NaN  
3          ...   NaN  NaN  µg/m³  NaN   NaN NaN NaN  NaN   84.9 NaN  
4          ...   NaN  NaN  µg/m³  NaN   NaN NaN NaN  NaN   36.8 NaN  

[5 rows x 22 columns]

In [ ]:
display(pivoted_df.head())
display(pivoted_df.info())

location_id sensors_id                 location  \
parameter                                                   
0                  12         23  SPARTAN - IIT Kanpur-12   
1                  12         23  SPARTAN - IIT Kanpur-12   
2                  12         23  SPARTAN - IIT Kanpur-12   
3                  12         23  SPARTAN - IIT Kanpur-12   
4                  12         23  SPARTAN - IIT Kanpur-12   

                           datetime     lat     lon  year month day      time  \
parameter                                                                       
0         2013-12-14 16:00:00+05:30  26.519  80.233  2013    12  14  16:00:00   
1         2013-12-14 17:00:00+05:30  26.519  80.233  2013    12  14  17:00:00   
2         2013-12-14 18:00:00+05:30  26.519  80.233  2013    12  14  18:00:00   
3         2013-12-14 19:00:00+05:30  26.519  80.233  2013    12  14  19:00:00   
4         2013-12-14 20:00:00+05:30  26.519  80.233  2013    12  14  20:00:00   

           ... units                 value                             
parameter  ...    o3 pm10   pm25 so2    co  no2   o3 pm10   pm25  so2  
0          ...             µg/m³       0.0  0.0  0.0  0.0  106.5  0.0  
1          ...             µg/m³       0.0  0.0  0.0  0.0  127.6  0.0  
2          ...             µg/m³       0.0  0.0  0.0  0.0  124.0  0.0  
3          ...             µg/m³       0.0  0.0  0.0  0.0   84.9  0.0  
4          ...             µg/m³       0.0  0.0  0.0  0.0   36.8  0.0  

[5 rows x 22 columns]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14841314 entries, 0 to 14841313
Data columns (total 22 columns):
 #   Column           Dtype                    
---  ------           -----                    
 0   (location_id, )  int64                    
 1   (sensors_id, )   int64                    
 2   (location, )     object                   
 3   (datetime, )     datetime64[ns, UTC+05:30]
 4   (lat, )          float64                  
 5   (lon, )          float64                  
 6   (year, )         int32                    
 7   (month, )        int32                    
 8   (day, )          int32                    
 9   (time, )         object                   
 10  (units, co)      object                   
 11  (units, no2)     object                   
 12  (units, o3)      object                   
 13  (units, pm10)    object                   
 14  (units, pm25)    object                   
 15  (units, so2)     object                   
 16  (value, co)     

None

In [ ]:
# Fill missing values appropriately based on column data type
for col in pivoted_df.columns:
    if pivoted_df[col].dtype == 'object':
        # Fill object type columns (like units) with an empty string
        pivoted_df[col] = pivoted_df[col].fillna('').astype(str) # Explicitly convert to string
    else:
        # Fill numeric columns (like values) with 0
        pivoted_df[col] = pivoted_df[col].fillna(0)


# Display the head and null counts to verify
display(pivoted_df.head())
display(pivoted_df.isnull().sum())

location_id sensors_id                 location  \
parameter                                                   
0                  12         23  SPARTAN - IIT Kanpur-12   
1                  12         23  SPARTAN - IIT Kanpur-12   
2                  12         23  SPARTAN - IIT Kanpur-12   
3                  12         23  SPARTAN - IIT Kanpur-12   
4                  12         23  SPARTAN - IIT Kanpur-12   

                           datetime     lat     lon  year month day      time  \
parameter                                                                       
0         2013-12-14 16:00:00+05:30  26.519  80.233  2013    12  14  16:00:00   
1         2013-12-14 17:00:00+05:30  26.519  80.233  2013    12  14  17:00:00   
2         2013-12-14 18:00:00+05:30  26.519  80.233  2013    12  14  18:00:00   
3         2013-12-14 19:00:00+05:30  26.519  80.233  2013    12  14  19:00:00   
4         2013-12-14 20:00:00+05:30  26.519  80.233  2013    12  14  20:00:00   

           ... units                 value                             
parameter  ...    o3 pm10   pm25 so2    co  no2   o3 pm10   pm25  so2  
0          ...             µg/m³       0.0  0.0  0.0  0.0  106.5  0.0  
1          ...             µg/m³       0.0  0.0  0.0  0.0  127.6  0.0  
2          ...             µg/m³       0.0  0.0  0.0  0.0  124.0  0.0  
3          ...             µg/m³       0.0  0.0  0.0  0.0   84.9  0.0  
4          ...             µg/m³       0.0  0.0  0.0  0.0   36.8  0.0  

[5 rows x 22 columns]

,,0
,parameter,
location_id,,0
sensors_id,,0
location,,0
datetime,,0
lat,,0
lon,,0
year,,0
month,,0
day,,0


In [ ]:
# Save the pivoted DataFrame to a Feather file as an alternative
output_path = "/content/drive/MyDrive/pivoted_openaq_data.feather"
try:
    pivoted_df.to_feather(output_path)
    print(f"Successfully saved pivoted data to {output_path}")
except Exception as e:
    print(f"Error saving pivoted data to {output_path}: {e}")

Successfully saved pivoted data to /content/drive/MyDrive/pivoted_openaq_data.feather


In [ ]:
# Check the timezone of the datetime column and get unique timezones
timezone_info = combined_df['datetime'].dt.tz

# Check if timezone_info is a single timezone object or a Series of timezone objects
if isinstance(timezone_info, object) and isinstance(timezone_info.tzname(None), str):
    # It's a single timezone object
    unique_timezones = [timezone_info]
    num_unique_timezones = 1
else:
    # It's likely a Series of timezone objects
    unique_timezones = timezone_info.unique()
    num_unique_timezones = len(unique_timezones)


print(f"There are {num_unique_timezones} unique timezones in the 'datetime' column.")
print("The unique timezones are:")
print(unique_timezones)

There are 1 unique timezones in the 'datetime' column.
The unique timezones are:
[datetime.timezone(datetime.timedelta(seconds=19800))]


In [ ]:
%pip install requests

## Fetch weather data for unique combinations

### Subtask:
Retry fetching historical weather data from the Open-Meteo API for the unique location-time combinations, implementing a delay to avoid hitting the rate limit.


In [ ]:
import pandas as pd

output_path = "/content/drive/MyDrive/pivoted_openaq_data.feather"

try:
    pivoted_df = pd.read_feather(output_path)
    print(f"Successfully loaded data from {output_path}")
    display(pivoted_df.head())
except Exception as e:
    print(f"Error loading data from {output_path}: {e}")

Successfully loaded data from /content/drive/MyDrive/pivoted_openaq_data.feather


location_id sensors_id                 location  \
parameter                                                   
0                  12         23  SPARTAN - IIT Kanpur-12   
1                  12         23  SPARTAN - IIT Kanpur-12   
2                  12         23  SPARTAN - IIT Kanpur-12   
3                  12         23  SPARTAN - IIT Kanpur-12   
4                  12         23  SPARTAN - IIT Kanpur-12   

                           datetime     lat     lon  year month day      time  \
parameter                                                                       
0         2013-12-14 16:00:00+05:30  26.519  80.233  2013    12  14  16:00:00   
1         2013-12-14 17:00:00+05:30  26.519  80.233  2013    12  14  17:00:00   
2         2013-12-14 18:00:00+05:30  26.519  80.233  2013    12  14  18:00:00   
3         2013-12-14 19:00:00+05:30  26.519  80.233  2013    12  14  19:00:00   
4         2013-12-14 20:00:00+05:30  26.519  80.233  2013    12  14  20:00:00   

           ... units                 value                             
parameter  ...    o3 pm10   pm25 so2    co  no2   o3 pm10   pm25  so2  
0          ...             µg/m³       0.0  0.0  0.0  0.0  106.5  0.0  
1          ...             µg/m³       0.0  0.0  0.0  0.0  127.6  0.0  
2          ...             µg/m³       0.0  0.0  0.0  0.0  124.0  0.0  
3          ...             µg/m³       0.0  0.0  0.0  0.0   84.9  0.0  
4          ...             µg/m³       0.0  0.0  0.0  0.0   36.8  0.0  

[5 rows x 22 columns]

In [ ]:
# Get unique location-year pairs
unique_loc_year = pivoted_df[['lat','lon','year']].drop_duplicates()
print("Total unique (lat,lon,year) pairs:", len(unique_loc_year))

Total unique (lat,lon,year) pairs: 392


In [ ]:
import requests
import os
import pandas as pd
import time

def fetch_weather(lat, lon, year, save_dir="/content/drive/MyDrive/weather_data", max_retries=5):
    """Fetch weather for a given lat, lon, year and save as parquet with retries."""
    lat = float(lat)
    lon = float(lon)
    year = int(year)

    os.makedirs(save_dir, exist_ok=True)
    outfile = f"{save_dir}/weather_{lat}_{lon}_{year}.parquet"

    if os.path.exists(outfile):
        print(f"Skipping {lat},{lon},{year} (already exists)")
        return outfile

    start = f"{year}-01-01"
    end   = f"{year}-12-31"

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start,
        "end_date": end,
        "hourly": ["temperature_2m", "windspeed_10m", "relative_humidity_2m"],
        "timezone": "Asia/Kolkata"
    }

    retries = 0
    while retries < max_retries:
        try:
            r = requests.get("https://archive-api.open-meteo.com/v1/archive", params=params, timeout=60)
            r.raise_for_status()
            data = r.json()

            if "hourly" not in data:
                print(f"No data for {lat},{lon},{year}")
                return None

            weather_df = pd.DataFrame({
                "datetime": pd.to_datetime(data["hourly"]["time"]),
                "temperature": data["hourly"]["temperature_2m"],
                "wind": data["hourly"]["windspeed_10m"],
                "humidity": data["hourly"]["relative_humidity_2m"]
            })
            weather_df["lat"] = lat
            weather_df["lon"] = lon

            weather_df.to_parquet(outfile, index=False)
            print(f"Saved {outfile}")
            return outfile

        except requests.exceptions.RequestException as e:
            retries += 1
            wait_time = 2 ** retries   # exponential backoff (2,4,8,16,32 sec)
            print(f"Error fetching {lat},{lon},{year} ({e}), retry {retries}/{max_retries} in {wait_time}s...")
            time.sleep(wait_time)

    print(f"Failed after {max_retries} retries for {lat},{lon},{year}")
    return None


In [ ]:
from tqdm import tqdm

for row in tqdm(unique_loc_year.itertuples(index=False), total=len(unique_loc_year)):
    # Access elements by index instead of attribute name
    lat = float(row[0])
    lon = float(row[1])
    year = int(row[2])

    fetch_weather(lat, lon, year)

  0%|          | 1/392 [00:01<06:46,  1.04s/it]

Saved /content/drive/MyDrive/weather_data/weather_26.519_80.233_2013.parquet


  1%|          | 2/392 [00:01<06:22,  1.02it/s]

Saved /content/drive/MyDrive/weather_data/weather_26.519_80.233_2014.parquet


  1%|          | 3/392 [00:03<06:32,  1.01s/it]

...

Saved /content/drive/MyDrive/weather_data/weather_17.460103_78.334361_2020.parquet


100%|█████████▉| 391/392 [09:09<00:01,  1.04s/it]

Saved /content/drive/MyDrive/weather_data/weather_17.460103_78.334361_2021.parquet


100%|██████████| 392/392 [09:10<00:00,  1.40s/it]

Saved /content/drive/MyDrive/weather_data/weather_17.460103_78.334361_2022.parquet


In [ ]:
import glob
import pandas as pd

# Load all saved weather data
files = glob.glob("/content/drive/MyDrive/weather_data/*.parquet")
weather_all = pd.concat([pd.read_parquet(f) for f in files], ignore_index=True)

# Save the combined weather data to a parquet file
output_path = "/content/drive/MyDrive/combined_weather_data.parquet"
try:
    weather_all.to_parquet(output_path, index=False)
    print(f"Successfully saved combined weather data to {output_path}")
except Exception as e:
    print(f"Error saving combined weather data: {e}")

display(weather_all.head())
display(weather_all.info())

Successfully saved combined weather data to /content/drive/MyDrive/combined_weather_data.parquet


,datetime,temperature,wind,humidity,lat,lon
0,2013-01-01 00:00:00,10.9,8.4,81,26.519,80.233
1,2013-01-01 01:00:00,10.2,8.3,82,26.519,80.233
2,2013-01-01 02:00:00,9.6,7.6,84,26.519,80.233
3,2013-01-01 03:00:00,9.0,6.3,86,26.519,80.233
4,2013-01-01 04:00:00,8.5,6.2,87,26.519,80.233


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3436248 entries, 0 to 3436247
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   datetime     datetime64[ns]
 1   temperature  float64       
 2   wind         float64       
 3   humidity     int64         
 4   lat          float64       
 5   lon          float64       
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 157.3 MB


None

In [ ]:
import pandas as pd
weather_all = pd.read_parquet("/content/drive/MyDrive/combined_weather_data.parquet")
weather_all.head()
weather_all.shape

(3436248, 6)

In [ ]:
weather_all.head()

,datetime,temperature,wind,humidity,lat,lon,datetime_hour,lat_round,lon_round
0,2013-01-01 00:00:00,10.9,8.4,81,26.519,80.233,2013-01-01 00:00:00,26.519,80.233
1,2013-01-01 01:00:00,10.2,8.3,82,26.519,80.233,2013-01-01 01:00:00,26.519,80.233
2,2013-01-01 02:00:00,9.6,7.6,84,26.519,80.233,2013-01-01 02:00:00,26.519,80.233
3,2013-01-01 03:00:00,9.0,6.3,86,26.519,80.233,2013-01-01 03:00:00,26.519,80.233
4,2013-01-01 04:00:00,8.5,6.2,87,26.519,80.233,2013-01-01 04:00:00,26.519,80.233


In [ ]:
pivoted_df.head()

location_id sensors_id                 location            datetime  \
parameter                                                                       
0                  12         23  SPARTAN - IIT Kanpur-12 2013-12-14 16:00:00   
1                  12         23  SPARTAN - IIT Kanpur-12 2013-12-14 17:00:00   
2                  12         23  SPARTAN - IIT Kanpur-12 2013-12-14 18:00:00   
3                  12         23  SPARTAN - IIT Kanpur-12 2013-12-14 19:00:00   
4                  12         23  SPARTAN - IIT Kanpur-12 2013-12-14 20:00:00   

              lat     lon  year month day      time  ... units value       \
parameter                                            ...   so2    co  no2   
0          26.519  80.233  2013    12  14  16:00:00  ...         0.0  0.0   
1          26.519  80.233  2013    12  14  17:00:00  ...         0.0  0.0   
2          26.519  80.233  2013    12  14  18:00:00  ...         0.0  0.0   
3          26.519  80.233  2013    12  14  19:00:00  ...         0.0  0.0   
4          26.519  80.233  2013    12  14  20:00:00  ...         0.0  0.0   

                                      datetime_hour lat_round lon_round  
parameter   o3 pm10   pm25  so2                                          
0          0.0  0.0  106.5  0.0 2013-12-14 16:00:00    26.519    80.233  
1          0.0  0.0  127.6  0.0 2013-12-14 17:00:00    26.519    80.233  
2          0.0  0.0  124.0  0.0 2013-12-14 18:00:00    26.519    80.233  
3          0.0  0.0   84.9  0.0 2013-12-14 19:00:00    26.519    80.233  
4          0.0  0.0   36.8  0.0 2013-12-14 20:00:00    26.519    80.233  

[5 rows x 25 columns]

In [ ]:
# Check for data completeness per hour per day per month per year for each latitude
weather_all['year'] = weather_all['datetime'].dt.year
weather_all['month'] = weather_all['datetime'].dt.month
weather_all['day'] = weather_all['datetime'].dt.day
weather_all['hour'] = weather_all['datetime'].dt.hour

completeness_check = weather_all.groupby(['lat', 'year', 'month', 'day'])['hour'].nunique()

# Identify combinations with less than 24 hours of data
incomplete_days = completeness_check[completeness_check < 24]

if not incomplete_days.empty:
    print("Incomplete data found for the following location-year-month-day combinations (less than 24 hours):")
    display(incomplete_days)
else:
    print("Data seems to be complete for all location-year-month-day combinations.")

# Further check: number of unique datetimes per location-year
datetime_completeness_check = weather_all.groupby(['lat', 'lon', 'year'])['datetime'].nunique()

# Total expected hours in a year (considering leap years)
# This is a simplified check and might not be perfectly accurate for all years
expected_hours = weather_all.groupby('year')['datetime'].apply(lambda x: (x.max() - x.min()).days * 24 if not x.empty else 0)

print("\nNumber of unique datetimes per location-year:")
display(datetime_completeness_check)

# Compare with expected hours (simplified)
print("\nComparison with expected hours (simplified):")

# Reset the index of datetime_completeness_check to align with expected_hours
comparison_df = datetime_completeness_check.reset_index()
comparison_df = comparison_df.rename(columns={'datetime': 'unique_hours'})

# Merge with expected_hours
comparison = comparison_df.merge(
    expected_hours.rename('expected_hours_approx').reset_index(),
    on='year',
    how='left'
)

display(comparison)

Data seems to be complete for all location-year-month-day combinations.

Number of unique datetimes per location-year:


lat        lon        year
12.909916  80.107654  2018    8760
                      2019    8760
                      2020    8784
                      2021    8760
                      2022    8760
                              ... 
31.620000  74.876512  2018    8760
                      2019    8760
                      2020    8784
                      2021    8760
                      2022    8760
Name: datetime, Length: 392, dtype: int64


Comparison with expected hours (simplified):


,lat,lon,year,unique_hours,expected_hours_approx
0,12.909916,80.107654,2018,8760,8736
1,12.909916,80.107654,2019,8760,8736
2,12.909916,80.107654,2020,8784,8760
3,12.909916,80.107654,2021,8760,8736
4,12.909916,80.107654,2022,8760,8736
...,...,...,...,...,...
387,31.620000,74.876512,2018,8760,8736
388,31.620000,74.876512,2019,8760,8736
389,31.620000,74.876512,2020,8784,8760
390,31.620000,74.876512,2021,8760,8736


In [ ]:
for lat in weather_all['lon'].unique():
    if lat not in pivoted_df['lon'].unique():
        print(f"Missing lat: {lat}")

In [ ]:
import pandas as pd

# Normalize datetime (remove timezone info to match)
pivoted_df['datetime'] = pivoted_df['datetime'].dt.tz_localize(None)
weather_all['datetime'] = weather_all['datetime'].dt.tz_localize(None)

pivoted_df['datetime_hour'] = pivoted_df['datetime'].dt.floor('h')
weather_all['datetime_hour'] = weather_all['datetime'].dt.floor('h')

pivoted_df['lat_round'] = pivoted_df['lat'].round(3)
pivoted_df['lon_round'] = pivoted_df['lon'].round(3)
weather_all['lat_round'] = weather_all['lat'].round(3)
weather_all['lon_round'] = weather_all['lon'].round(3)

# Remove index from both DataFrames
pivoted_df = pivoted_df.reset_index(drop=True)
weather_all = weather_all.reset_index(drop=True)

# Flatten the MultiIndex columns of pivoted_df
pivoted_df.columns = ['_'.join(str(s) for s in col if s) for col in pivoted_df.columns]


# Perform the left join
merged_df = pivoted_df.merge(
    weather_all[['datetime_hour', 'lat_round', 'lon_round', 'temperature', 'wind', 'humidity']],
    on=['datetime_hour', 'lat_round', 'lon_round'],
    how='left'
)

# Display the head of the merged DataFrame
display(merged_df.head())
display(merged_df.info())

,location_id,sensors_id,location,datetime,lat,lon,year,month,day,time,...,value_o3,value_pm10,value_pm25,value_so2,datetime_hour,lat_round,lon_round,temperature,wind,humidity
0,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 16:00:00,26.519,80.233,2013,12,14,16:00:00,...,0.0,0.0,106.5,0.0,2013-12-14 16:00:00,26.519,80.233,22.2,3.8,40
1,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 17:00:00,26.519,80.233,2013,12,14,17:00:00,...,0.0,0.0,127.6,0.0,2013-12-14 17:00:00,26.519,80.233,20.6,7.2,46
2,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 18:00:00,26.519,80.233,2013,12,14,18:00:00,...,0.0,0.0,124.0,0.0,2013-12-14 18:00:00,26.519,80.233,18.8,9.0,52
3,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 19:00:00,26.519,80.233,2013,12,14,19:00:00,...,0.0,0.0,84.9,0.0,2013-12-14 19:00:00,26.519,80.233,17.2,9.4,59
4,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 20:00:00,26.519,80.233,2013,12,14,20:00:00,...,0.0,0.0,36.8,0.0,2013-12-14 20:00:00,26.519,80.233,15.8,8.2,65


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14841314 entries, 0 to 14841313
Data columns (total 28 columns):
 #   Column         Dtype         
---  ------         -----         
 0   location_id    int64         
 1   sensors_id     int64         
 2   location       object        
 3   datetime       datetime64[ns]
 4   lat            float64       
 5   lon            float64       
 6   year           int32         
 7   month          int32         
 8   day            int32         
 9   time           object        
 10  units_co       object        
 11  units_no2      object        
 12  units_o3       object        
 13  units_pm10     object        
 14  units_pm25     object        
 15  units_so2      object        
 16  value_co       float64       
 17  value_no2      float64       
 18  value_o3       float64       
 19  value_pm10     float64       
 20  value_pm25     float64       
 21  value_so2      float64       
 22  datetime_hour  datetime64[ns]
 23  lat_r

None

In [ ]:
display(merged_df.isnull().sum())

,0
location_id,0
sensors_id,0
location,0
datetime,0
lat,0
lon,0
year,0
month,0
day,0
time,0


In [ ]:
output_path = "/content/drive/MyDrive/air_quality_with_weather.parquet"

try:
    merged_df.to_parquet(output_path, index=False)
    print(f"Successfully saved merged data to {output_path}")
except Exception as e:
    print(f"Error saving merged data to {output_path}: {e}")

Successfully saved merged data to /content/drive/MyDrive/air_quality_with_weather.parquet


In [ ]:
import pandas as pd
combined = pd.read_parquet("/content/drive/MyDrive/air_quality_with_weather.parquet")
combined.head()
combined.shape

(14841314, 28)

In [ ]:
# Conversion factor for O3 from ppm to µg/m³ at 25°C and 1 atm
# 1 ppm O3 = 1.96 µg/m³ O3
conversion_factor_o3 = 1.96

# Identify rows where units_o3 is 'ppm'
ppm_o3_mask = combined['units_o3'] == 'ppm'

# Convert values from ppm to µg/m³ for these rows
combined.loc[ppm_o3_mask, 'value_o3'] = combined.loc[ppm_o3_mask, 'value_o3'] * conversion_factor_o3

# Update the units to 'µg/m³' for these rows
combined.loc[ppm_o3_mask, 'units_o3'] = 'µg/m³'

# Verify the changes
print("Unique values for 'units_o3' after conversion:")
display(combined['units_o3'].unique())

print("\nHead of DataFrame after conversion:")
display(combined[combined['units_o3'] == 'µg/m³'].head())

Unique values for 'units_o3' after conversion:


array(['', 'µg/m³'], dtype=object)


Head of DataFrame after conversion:


,location_id,sensors_id,location,datetime,lat,lon,year,month,day,time,...,value_o3,value_pm10,value_pm25,value_so2,datetime_hour,lat_round,lon_round,temperature,wind,humidity
1684,13,24,"Delhi Technological University, Delhi - CPCB-13",2015-04-09 11:30:00,28.7440,77.1200,2015,4,9,11:30:00,...,2.00,0.0,0.0,0.0,2015-04-09 11:00:00,28.744,77.120,28.0,12.7,42
47091,15,29,IGI Airport-15,2015-07-10 12:00:00,28.5600,77.0940,2015,7,10,12:00:00,...,61.66,0.0,0.0,0.0,2015-07-10 12:00:00,28.560,77.094,27.1,6.0,92
126199,17,394,"R K Puram, Delhi - DPCC-17",2015-06-29 20:00:00,28.5648,77.1744,2015,6,29,20:00:00,...,24.20,0.0,0.0,0.0,2015-06-29 20:00:00,28.565,77.174,30.2,10.4,63
126200,17,394,"R K Puram, Delhi - DPCC-17",2015-06-30 12:40:00,28.5648,77.1744,2015,6,30,12:40:00,...,147.20,0.0,0.0,0.0,2015-06-30 12:00:00,28.565,77.174,32.3,8.3,57
126201,17,394,"R K Puram, Delhi - DPCC-17",2015-06-30 18:40:00,28.5648,77.1744,2015,6,30,18:40:00,...,25.50,0.0,0.0,0.0,2015-06-30 18:00:00,28.565,77.174,29.5,13.1,66


In [ ]:
# List of unit columns to drop
units_to_drop = [
    'units_co',
    'units_no2',
    'units_o3',
    'units_pm10',
    'units_pm25',
    'units_so2'
]

# Drop the individual unit columns
combined = combined.drop(columns=units_to_drop)

# Create a single 'units' column with 'µg/m³'
combined['units'] = 'µg/m³'

# Display the head and info to verify the changes
display(combined.head())
display(combined.info())

,location_id,sensors_id,location,datetime,lat,lon,year,month,day,time,...,value_pm10,value_pm25,value_so2,datetime_hour,lat_round,lon_round,temperature,wind,humidity,units
0,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 16:00:00,26.519,80.233,2013,12,14,16:00:00,...,0.0,106.5,0.0,2013-12-14 16:00:00,26.519,80.233,22.2,3.8,40,µg/m³
1,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 17:00:00,26.519,80.233,2013,12,14,17:00:00,...,0.0,127.6,0.0,2013-12-14 17:00:00,26.519,80.233,20.6,7.2,46,µg/m³
2,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 18:00:00,26.519,80.233,2013,12,14,18:00:00,...,0.0,124.0,0.0,2013-12-14 18:00:00,26.519,80.233,18.8,9.0,52,µg/m³
3,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 19:00:00,26.519,80.233,2013,12,14,19:00:00,...,0.0,84.9,0.0,2013-12-14 19:00:00,26.519,80.233,17.2,9.4,59,µg/m³
4,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 20:00:00,26.519,80.233,2013,12,14,20:00:00,...,0.0,36.8,0.0,2013-12-14 20:00:00,26.519,80.233,15.8,8.2,65,µg/m³


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14841314 entries, 0 to 14841313
Data columns (total 23 columns):
 #   Column         Dtype         
---  ------         -----         
 0   location_id    int64         
 1   sensors_id     int64         
 2   location       object        
 3   datetime       datetime64[ns]
 4   lat            float64       
 5   lon            float64       
 6   year           int32         
 7   month          int32         
 8   day            int32         
 9   time           object        
 10  value_co       float64       
 11  value_no2      float64       
 12  value_o3       float64       
 13  value_pm10     float64       
 14  value_pm25     float64       
 15  value_so2      float64       
 16  datetime_hour  datetime64[ns]
 17  lat_round      float64       
 18  lon_round      float64       
 19  temperature    float64       
 20  wind           float64       
 21  humidity       int64         
 22  units          object        
dtypes: da

None

In [ ]:
import pandas as pd
import numpy as np

# --- 1. Breakpoints Dictionary (Remains the same) ---
breakpoints = {
    "pm25": [(0, 30, 0, 50), (31, 60, 51, 100), (61, 90, 101, 200), (91, 120, 201, 300), (121, 250, 301, 400), (251, 9999, 401, 500)],
    "pm10": [(0, 50, 0, 50), (51, 100, 51, 100), (101, 250, 101, 200), (251, 350, 201, 300), (351, 430, 301, 400), (431, 9999, 401, 500)],
    "no2":  [(0, 40, 0, 50), (41, 80, 51, 100), (81, 180, 101, 200), (181, 280, 201, 300), (281, 400, 301, 400), (401, 9999, 401, 500)],
    "so2":  [(0, 40, 0, 50), (41, 80, 51, 100), (81, 380, 101, 200), (381, 800, 201, 300), (801, 1600, 301, 400), (1601, 9999, 401, 500)],
    "co":   [(0, 1, 0, 50), (1.1, 2, 51, 100), (2.1, 10, 101, 200), (10.1, 17, 201, 300), (17.1, 34, 301, 400), (34.1, 9999, 401, 500)],
    "o3":   [(0, 50, 0, 50), (51, 100, 51, 100), (101, 168, 101, 200), (169, 208, 201, 300), (209, 748, 301, 400), (749, 9999, 401, 500)]
}

# --- 2. The Core AQI Calculation Function (Remains the same) ---

def compute_aqi(cp, pollutant):
    """
    Computes the Air Quality Index (AQI) for a given pollutant concentration (Cp).
    """
    if cp is None or pd.isna(cp):
        return None

    # Ensure Cp is treated as a float for calculations
    cp = float(cp)

    # Check for division by zero risk (Bphi - Bplo == 0)
    for bp_lo, bp_hi, i_lo, i_hi in breakpoints[pollutant]:
        if bp_hi == bp_lo:
             continue # Skip invalid breakpoint ranges

        if bp_lo <= cp <= bp_hi:
            # The AQI formula: I = [(Ihi - Ilo) / (Bphi - Bplo)] * (Cp - Bplo) + Ilo
            return ((i_hi - i_lo) / (bp_hi - bp_lo)) * (cp - bp_lo) + i_lo

    return None


# --- 3. Pandas Implementation (Replaces Spark Transformations) ---


# List of pollutants and their corresponding DataFrame columns
pollutants = {
    "pm25": "value_pm25",
    "pm10": "value_pm10",
    "no2":  "value_no2",
    "so2":  "value_so2",
    "co":   "value_co",
    "o3":   "value_o3"
}

# 3a. Calculate individual AQI columns using .apply()
aqi_columns = []
for pol, col_name in pollutants.items():
    aqi_col_name = f"aqi_{pol}"

    # Use .apply() to execute the function row-wise on the concentration column.
    # The 'lambda' handles passing the pollutant name for the breakpoints lookup.
    df[aqi_col_name] = df[col_name].apply(lambda cp: compute_aqi(cp, pol))

    aqi_columns.append(aqi_col_name)

# 3b. Final AQI calculation (Replaces Spark's 'greatest')
# Use the .max(axis=1) method on the subset of AQI columns.
# skipna=True ensures that if one AQI value is None/NaN, the max of the others is taken.
df["AQI"] = df[aqi_columns].max(axis=1, skipna=True)

# print(df) # Uncomment to see the resulting DataFrame

In [2]:
import pandas as pd

df = pd.read_parquet('/content/drive/MyDrive/air_quality_with_weather1.parquet')
print(df.head())

   location_id  sensors_id                 location            datetime  \
0           12          23  SPARTAN - IIT Kanpur-12 2013-12-14 16:00:00   
1           12          23  SPARTAN - IIT Kanpur-12 2013-12-14 17:00:00   
2           12          23  SPARTAN - IIT Kanpur-12 2013-12-14 18:00:00   
3           12          23  SPARTAN - IIT Kanpur-12 2013-12-14 19:00:00   
4           12          23  SPARTAN - IIT Kanpur-12 2013-12-14 20:00:00   

      lat     lon  year  month  day      time  ...  wind  humidity  units  \
0  26.519  80.233  2013     12   14  16:00:00  ...   3.8        40  µg/m³   
1  26.519  80.233  2013     12   14  17:00:00  ...   7.2        46  µg/m³   
2  26.519  80.233  2013     12   14  18:00:00  ...   9.0        52  µg/m³   
3  26.519  80.233  2013     12   14  19:00:00  ...   9.4        59  µg/m³   
4  26.519  80.233  2013     12   14  20:00:00  ...   8.2        65  µg/m³   

     aqi_pm25  aqi_pm10  aqi_no2 aqi_so2  aqi_co  aqi_o3         AQI  
0  253.913793  

In [3]:
# List the columns you want to remove
columns_to_drop = [
    'aqi_pm25',
    'aqi_pm10',
    'aqi_no2',
    'aqi_so2',
    'aqi_co',
    'aqi_o3'
]

df.drop(columns=columns_to_drop, axis=1, inplace=True)

print("Individual AQI columns successfully dropped from df.")

Individual AQI columns successfully dropped from df.


In [4]:
df.head()

,location_id,sensors_id,location,datetime,lat,lon,year,month,day,time,...,value_pm25,value_so2,datetime_hour,lat_round,lon_round,temperature,wind,humidity,units,AQI
0,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 16:00:00,26.519,80.233,2013,12,14,16:00:00,...,106.5,0.0,2013-12-14 16:00:00,26.519,80.233,22.2,3.8,40,µg/m³,253.913793
1,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 17:00:00,26.519,80.233,2013,12,14,17:00:00,...,127.6,0.0,2013-12-14 17:00:00,26.519,80.233,20.6,7.2,46,µg/m³,306.065116
2,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 18:00:00,26.519,80.233,2013,12,14,18:00:00,...,124.0,0.0,2013-12-14 18:00:00,26.519,80.233,18.8,9.0,52,µg/m³,303.302326
3,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 19:00:00,26.519,80.233,2013,12,14,19:00:00,...,84.9,0.0,2013-12-14 19:00:00,26.519,80.233,17.2,9.4,59,µg/m³,182.589655
4,12,23,SPARTAN - IIT Kanpur-12,2013-12-14 20:00:00,26.519,80.233,2013,12,14,20:00:00,...,36.8,0.0,2013-12-14 20:00:00,26.519,80.233,15.8,8.2,65,µg/m³,60.800000


In [5]:
output_path = "/content/drive/MyDrive/air_quality_with_weather1.parquet"

try:
    df.to_parquet(output_path, index=False)
    print(f"Successfully saved merged data to {output_path}")
except Exception as e:
    print(f"Error saving merged data to {output_path}: {e}")

Successfully saved merged data to /content/drive/MyDrive/air_quality_with_weather1.parquet
